# CMORising ACCESS-ESM1.6 Spin-Up for ACCESS-ENSO-Recipes

This notebook is a cope from a tutorial in [ACCESS-MOPPeR](https://github.com/ACCESS-NRI/ACCESS-MOPPeR)

The ACCESS CMIP7 consortium is actively developing the ACCESS-ESM1.6 model for submission to the CMIP7 Fasttrack initiative. As part of this effort, a spin-up run is being conducted on the NCI Gadi supercomputer, with output data progressively becoming available. However, this data cannot yet be easily evaluated using established evaluation frameworks such as [ESMValTool](https://www.esmvaltool.org/) or [ILAMB](https://www.ilamb.org/).

This notebook demonstrates how to process the variables required by the [ACCESS-ENSO-Recipe](https://github.com/ACCESS-NRI/ACCESS-ENSO-recipes), developed by ACCESS-NRI and leveraging ESMValTool, to enable robust evaluation of ACCESS-ESM1.6 spin-up runs. This workflow provides a pathway for the scientific assessment and validation of the new model output.

## Variables Required for ACCESS-NRI ENSO Recipes

To run the [ACCESS-NRI ENSO Recipes](https://github.com/ACCESS-NRI/ACCESS-ENSO-recipes) on the current ACCESS-ESM1.6 spin-up (CMIP7 FastTrack), the following variables are needed:

- **pr**: Precipitation
- **tos**: Sea surface temperature (can be interchanged with `ts`)
- **tauu**: Zonal wind stress
- **ts**: Surface air temperature (can be interchanged with `tos`)
- **zos**: Sea surface height
- **nhf**: Net heat flux, calculated as  
    `nhf = (-hfls - hfss + rlds - rlus + rsds - rsus)`

**Notes:**

- The mapping for `rlus` and `rsus` is currently unknown.
- `ts` and `tos` can be used interchangeably.

This list is based on the requirements for the ENSO diagnostics and workflow as described in the ACCESS-NRI ENSO Recipes documentation.

To run the CMORIsation we advice using an X-Large or XX-Large session on ARE.

**Note:**  
ARE (Analysis Research Environment) is the interactive computing system provided by NCI (National Computational Infrastructure). Using a larger session (X-Large or XX-Large) ensures sufficient memory and CPU resources for efficient processing of large climate datasets.

In [1]:
from access_mopper import ACCESS_ESM_CMORiser


Loaded Configuration:
Creator Name: felicity Chun
Organisation: ACCESS-NRI
Creator Email: felicity.chun@anu.edu.au
Creator URL: 0009-0007-0845-0953


In [2]:
import dask.distributed as dask

client = dask.Client(threads_per_worker = 1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34837,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:34433,Total threads: 1
Dashboard: /proxy/46103/status,Memory: 4.47 GiB
Nanny: tcp://127.0.0.1:42123,


We are currently using the June Spin Up dataset located at:

`/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/`

This folder contains the output files from the ACCESS-ESM1.6 spin-up experiment, which are used throughout this notebook for CMORisation and evaluation.

In [3]:
ROOT_FOLDER = "/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/JuneSpinUp-JuneSpinUp-bfaa9c5b/"
# ROOT_FOLDER = "/scratch/p66/jxs599/access-esm/archive/AugustSpinUp-Jhan-dev-20250808-1-d1b0b669/"
ROOT_FOLDER = "/scratch/p66/jxs599/access-esm/archive/JuneSpinUp-JuneSpinUp-bfaa9c5b/"

In [4]:
OUTPUT_FOLDER = "/scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66" #"/scratch/iq82/fc6164/MOPPeR" 

In [5]:
# ! mkdir /scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66

In [7]:
import glob
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")

In [8]:
len(FILES), FILES[:5]


(6000,
 ['/scratch/p66/jxs599/access-esm/archive/JuneSpinUp-JuneSpinUp-bfaa9c5b/output076/atmosphere/netCDF/aiihca.pa-116802_mon.nc',
  '/scratch/p66/jxs599/access-esm/archive/JuneSpinUp-JuneSpinUp-bfaa9c5b/output076/atmosphere/netCDF/aiihca.pa-116803_mon.nc',
  '/scratch/p66/jxs599/access-esm/archive/JuneSpinUp-JuneSpinUp-bfaa9c5b/output076/atmosphere/netCDF/aiihca.pa-116812_mon.nc',
  '/scratch/p66/jxs599/access-esm/archive/JuneSpinUp-JuneSpinUp-bfaa9c5b/output076/atmosphere/netCDF/aiihca.pa-116809_mon.nc',
  '/scratch/p66/jxs599/access-esm/archive/JuneSpinUp-JuneSpinUp-bfaa9c5b/output076/atmosphere/netCDF/aiihca.pa-116801_mon.nc'])

In [5]:
parent_experiment_config = {
        "parent_experiment_id": "piControl",
        "parent_activity_id": "CMIP",
        "parent_source_id": "ACCESS-ESM1-5",
        "parent_variant_label": "r1i1p1f1",
        "parent_time_units": "days since 0001-01-01 00:00:00",
        "parent_mip_era": "CMIP6",
        "branch_time_in_child": 0.0,
        "branch_time_in_parent": 54786.0,
        "branch_method": "standard"
}

### Amon.pr

In [9]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [10]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.pr",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [11]:
cmoriser.run()

In [13]:
cube = cmoriser.to_iris()[0]
t_coord = cube.coord('time')
timels = [d for d in t_coord.units.num2date(t_coord.points)]
print(timels[0], timels[-1])

/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/fileformats/cf.py:664: IrisCfMissingVarWarning: Missing CF-netCDF grid mapping variable 'latitude_longitude', referenced by netCDF variable 'pr'
  warnings.warn(
/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/fileformats/cf.py:880: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'pr'
  warnings.warn(


1092-01-16 12:00:00 1591-12-16 12:00:00


/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/common/mixin.py:195: FutureWarning: You are using legacy date precision for Iris units - max precision is seconds. In future, Iris will use microsecond precision - available since cf-units version 3.3 - which may affect core behaviour. To opt-in to the new behaviour, set `iris.FUTURE.date_microseconds = True`.
  warnings.warn(message, category=FutureWarning)


In [11]:
import xarray as xr
ds = cmoriser.to_dataset()
for var, val in ds.data_vars.items():
    if var == 'pr':
        darray = val # Dask graph	6000 chunks in 12002 graph layers
        # save bnds? to apply to coords

# darray = darray.sortby('time') ## rechunk?
esm16_cube = darray.to_iris()

In [26]:
import access_mopper
print(access_mopper.__path__[0])

/home/189/fc6164/esmValTool/repos/ACCESS-MOPPeR/src/access_mopper


In [12]:
cmoriser.write() #write setting output folder

CMORised output written to /scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66/pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


In [17]:
import iris
fp = '/scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66/pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc'

MOPPeR
esmvaltool_output
esmvaltool_outputs
pcmdi_metrics
pr_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_136706-157204.nc
tmp


### Omon.tos

In [9]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/ocean/ocean-2d-surface_temp-1monthly-mean*.nc")
len(FILES)

500

In [10]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Omon.tos",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    parent_info=parent_experiment_config, # <-- This is optional, can be skipped if not needed
    output_path=OUTPUT_FOLDER
    )

In [11]:
cmoriser.run()

In [12]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66/tos_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


### Amon.tauu

In [13]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [14]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.tauu",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [15]:
cmoriser.run()

In [ ]:
# type(cmoriser)
# del cmoriser, #cancel(futures)
client.restart()

In [16]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66/tauu_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


### Amon.ts

In [17]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [18]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.ts",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5", #controlled variable
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [19]:
cmoriser.run()

In [20]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66/ts_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


In [21]:
# !ls /scratch/iq82/fc6164/MOPPeR #'/scratch/p66/fc6164/MOPPeR/tauu_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc'
import iris
fp = '/scratch/p66/fc6164/MOPPeR/JuneSpinUp-p66/ts_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc'  
# '/scratch/p66/fc6164/MOPPeR/ts_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc'
iris.load(fp)

/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/fileformats/cf.py:664: IrisCfMissingVarWarning: Missing CF-netCDF grid mapping variable 'latitude_longitude', referenced by netCDF variable 'ts'
  warnings.warn(
/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/fileformats/cf.py:880: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'ts'
  warnings.warn(


[<iris 'Cube' of surface_temperature / (K) (longitude: 192; latitude: 145; time: 6000)>]

### Omon.zos

In [21]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/ocean/ocean-2d-sea_level-1monthly-mean*.nc")
len(FILES)

500

In [22]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Omon.zos",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [23]:
cmoriser.run()

In [24]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/zos_Omon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


### Amon.hfss

In [25]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [26]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.hfss",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [27]:
cmoriser.run()

In [28]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/hfss_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


### Amon.hfls

In [29]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.hfls",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [30]:
cmoriser.run()

In [31]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/hfls_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


### Amon.rlds

In [32]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [33]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rlds",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [34]:
cmoriser.run()

In [35]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/rlds_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


### Amon.rlus

In [22]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [23]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rlus",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [24]:
cmoriser.run()

In [26]:
cmoriser.write()

/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 21.46 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Exception ignored in: <function CachingFileManager.__del__ at 0x148ea848f240>
Traceback (most recent call last):
  File "/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2618, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2585, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2113, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID


CMORised output written to /scratch/p66/fc6164/MOPPeR/rlus_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


In [25]:
cmoriser.to_iris()

/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/fileformats/cf.py:880: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'rlus'
  warnings.warn(


[<iris 'Cube' of surface_upwelling_longwave_flux_in_air / (W m-2) (longitude: 192; latitude: 145; time: 6000)>]

In [ ]:
# client.close()

### Amon.rsds

In [36]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [37]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rsds",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [38]:
cmoriser.run()

In [39]:
cmoriser.write()

CMORised output written to /scratch/p66/fc6164/MOPPeR/rsds_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


### Amon.rsus

In [18]:
FILES = glob.glob(ROOT_FOLDER + "output[0-4][0-9][0-9]/atmosphere/netCDF/*mon.nc")
len(FILES)

6000

In [19]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.rsus",
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [20]:
cmoriser.run()

In [21]:
cmoriser.write()

/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.49 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Exception ignored in: <function CachingFileManager.__del__ at 0x1545f187a020>
Traceback (most recent call last):
  File "/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2618, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2585, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2113, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID


CMORised output written to /scratch/p66/fc6164/MOPPeR/rsus_Amon_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_109201-159112.nc


In [31]:
cmoriser = ACCESS_ESM_CMORiser(
    input_paths=FILES,
    compound_name="Amon.clt", #KeyError: 'alevel' cl
    experiment_id="piControl",
    source_id="ACCESS-ESM1-5",
    variant_label="r1i1p1f1",
    grid_label="gn",
    activity_id="CMIP",
    output_path=OUTPUT_FOLDER,
    parent_info=parent_experiment_config # <-- This is optional, can be skipped if not needed
    )

In [32]:
cmoriser.run()

In [33]:
cmoriser.to_iris()

/g/data/tm70/fc6164/conda/envs/esmvaltool_dev/lib/python3.11/site-packages/iris/fileformats/cf.py:880: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'clt'
  warnings.warn(


[<iris 'Cube' of cloud_area_fraction / (%) (longitude: 192; latitude: 145; time: 6000)>]

In [30]:
import iris
iris.load(FILES[0])[107]

<iris 'Cube' of cloud_area_fraction / (1) (time: 1; latitude: 145; longitude: 192)>